# PS 88 Lab 10: Instruments and Lotteries

In [ ]:
import numpy as np 
import pandas as pd 
import statsmodels.api as sm 
import statsmodels.formula.api as smf 
from itertools import combinations 
import seaborn as sns
from linearmodels.iv import IV2SLS

## Part 1. Building on a Class Example

Let's do a simulation of the example I gave in class, where I flip a coin for each student in a class, and for those who get heads I give them `amount=30` dollars if they attend the next lecture. Imagine a researcher does not know the exact amount I gave, and tries to figure it out by comparing how much cash everyone has in their wallet afterwards.

First let's define the amount I propose to give, and randomly generate the `baseline` amount of money-in-wallet held by a large class (2000 students). Assume this is uniformly distrubuted between 0 and 100. We store the result of the coinflip with the variable `ittreat` ("intention to treat"), equal to 1 for those who I will give `amount` to if they show up to class.


In [ ]:
amount = 30
nstudent = 2000
baseline = 100*np.random.rand(nstudent)
ittreat = 1*(np.random.rand(nstudent) > .5)

Let's make an extreme assumption that the only people who show up to class are those with below average baseline money-in-wallet (maybe they think they have better things to do). 

Let `showup` be equal to 1 for those who show up to lecture and 0 for those who do not. Those who get the money are those with `showup=1` and `ittreat=1`. So, we can compute the actual treatment (receiving `amount`) by multiplying `showup` times `ittreat`. The realized money-in-wallet is then equal to `baseline + amount` for those who get treated, and  `baseline` otherwise. 

In [ ]:
showup = 1*(baseline < 50)
treat = showup*ittreat
realized = np.where(treat, baseline + amount, baseline)

Now let's suppose the resarcher has access to a dataset which tells them (1) the student coinflip (`ittreat`, or the "instrument"), (2) whether they get treated, and (3) their realized money-in-wallet.

In [ ]:
obs = pd.DataFrame(data={'instrument': ittreat, 
                             'treat': treat, 
                             'wallet': realized})
obs

We can the compute the "first stage" as the difference in receiving the treatment among those who got heads on their coinflip (`obs['instrument']==1`) vs not (`obs['instrument']==0`).

We can also compute the "reduced form" which is the difference in outcome (`wallet`) between those who got heads on their coinflip vs not.

In [ ]:
# First stage stage
phi = (np.mean(obs.loc[obs['instrument']==1, 'treat']) - 
       np.mean(obs.loc[obs['instrument']==0, 'treat']))
# Reduced form
rho = (np.mean(obs.loc[obs['instrument']==1, 'wallet']) - 
       np.mean(obs.loc[obs['instrument']==0, 'wallet']))

print(phi,rho)

**Question 1.1. Use the formula for the Local Average Treatment Effect to recover the causal effect of the treatment.**

In lecture I said we can assume that the baseline money of those with a heads coin flip will be pretty similar to those who get a tails coin flip. Let's check this:

In [ ]:
np.mean(baseline[ittreat == 1]) - np.mean(baseline[ittreat == 0])

**Question 1.2. Now compute a "naive" difference of means between the amount of money-in-wallet among those who received the treatment versus not.**

**Question 1.3. Among your answers to 1.1 and 1.2, one of them should be close to the real `amount` given in this simulation. Why is this answer a good estimate while the other is not?**

*Answer to 1.3*

*Note: the rest of this part is optional.* Now let's consider an alternative version of the simulation where only those who have more money-in-pocket at the baseline show up to class.

In [ ]:
showup2 = baseline > 50
treat2 = showup2*ittreat
realized2 = np.where(treat2, baseline + amount, baseline)
obs2 = pd.DataFrame(data={'instrument': ittreat, 
                             'treat': treat2, 
                             'wallet': realized2})
obs2

**Question 1.4 [Optional]. Compute the first stage, reduced form, and local average treatment effect, and "naive" difference of means for these data. Compare your answers to what you got in 1.1-1.3.**

## Part 2. Health Insurance, Revisited

Section 1.2 of Mastering Metrics contains a discussion of the "Oregon Health Experiment (OHP)", where Oregon decided to expand acces to Medicaid (health insurance for those with low incomes) by lottery. 75,000 people were entered in the lottery, and 30,000 won, meaning they were invited to apply to the program, but still had to do so on time and meet other eligibility requirements. In the end, 40% of the winners received coverage. The lottery losers were also sometimes able to get access to coverage as well, and 14% did so.

Here are some differences in average outcomes for lottery winners and losers (from Table 1.5; some of these come from a sample of just the Portland area but we can set this difference aside):
- Winners had an average of 0.1 more visits to an emergency room
- Winners were 3.9% more likely to say their "health is good"
- Winners had a higher average score of 0.29 on a 100 point scale for physical health
- Winners were 3% less likely to have medical debt

**Question 2.1. Compute the local average treatment effect of *getting Medicaid* for these four outcomes. Hint: use the difference in receiving Medicaid between lottery winners and losers to compute the "fist stage" (`phi`), and then use the four differences in winners/losers reported above as the the "reduced form" (you can call these `rho1`, `rho2`, etc.), and use the `rho/phi` forumula to compute the LATE for each outcome.**



One reason this study is so prominent is that it speaks to a huge debate in the United States about whether providing government insurance (like Medicaid) to all Americans would improve health outcomes/lower health care spending. 

**Question 2.2. Identify one of the results from part 2.1 that supports the argument that the government *should* provide more insurance, and one result that supports the argument that the government *should not* provide more insurance.**

Recall the LATE is for "compliers", i.e., people who get the treatment (here, Medicaid) if and only if they they get the intention to treat (here, winning the lottery). 

**Question 2.3. In this context, what is an "always taker"? Give a reason we might expect one or more of the causal effects of getting insurance to be different for an always taker than a complier.**

*Answer to 2.3*

## Part 3. A Political Lottery/When to *Not* Use 2SLS

A prominent example of a "real world" lottery in the political realm was the Vietnam draft lottery in 1969. You can read more about it <a href="https://en.wikipedia.org/wiki/Draft_lottery_(1969)">here</a>.

For our purposes, the key thing to know is that the draft lottery primarily affected men who (1) were born between 1944 and 1950, (2) had not yet enlisted in the military, and (3) had previously received a deferral due to being in college. The lottery itself gave a random "ranking" to all dates of birth, i.e., each DOB was given a lottery number from 1 to 366. (See what your draft number would have been <a href='http://history.hanover.edu/pictures2/1969-1202-madcour-p2-lottery.jpg'>here</a>!) Those who were born on a date with a lower number were more likely to be drafted, though whether this actually happened depended on several other factors. For example, some would have been ruled ineligible for service for medical reasons, or lived in places that already filled quotas for the number of men to enlist.

In a widely cited <a href="https://www.jstor.org/stable/41495063?seq=1#metadata_info_tab_contents">paper</a>, Berkeley's own Laura Stoker (and Robert Erikson) used a survey of men who were born in 1948 to see what the effect of the draft number was on attitudes towards the war and other political attitudes.

Let's pull up their replication data. (The `convert_categoricals=False` prevents an error related to some variables we won't work with.)

In [ ]:
viet = pd.read_stata("es_rep.dta", convert_categoricals=False)
viet

Recall we are primarily intested in respondents who had attended college and not yet served in the military. We can identify this subset as follows:

In [ ]:
colnomil = viet[(viet['cprep']==1) & (viet['milpre69'] == 0)]
colnomil

We are down to a relevant sample of 261, which isn't huge, but enough to work with.

The outcome variable we will consider is `vietx`, an index of pro-war attitudes which ranges from 0 (most anti-war) to 1 (most pro-war). It can take on values 0, .25,  .5, ,75, or 1.

Let's look at a histogram of this (setting bins=5 to make it look nicer).

In [ ]:
colnomil['vietx'].hist(bins=5)

**Question 3.1 Make a similar histogram using  the full sample (i.e., the `viet` dataframe rather than `colnomil`. How do the war attitudes differ among the full sample vs the college-attenders who did not  yet enlist?**

*Words for 3.1*

Let's do a version of the graph from the paper that looks at the relationship between draft number and the war attitude index.

In [ ]:
sns.regplot(x='lotnum', y='vietx', data=colnomil, ci=0)

As we an see, there is a postive relationship, meaning those with higher lottery numbers (less likely to be drafted) are more pro-war. We can also use `smf.ols` to produce this bivariate regression. 

In [ ]:
hawk_m1  = smf.ols('vietx ~ lotnum', data=colnomil).fit()
hawk_m1.summary()

We can compute the predicted change in going from the lowest draft number (1) to an average draft number (184) by multipling this difference times the slope parameter, which we can access with `hawk_m1.params`:

In [ ]:
hawk_m1.params[1]*(184  - 1)

**Question 3.2. What is the predicted shift in pro-war attitude when going from the lowest draft number (1) to the highest (366)?**

For a later regression, it will be helpful to use a version of the lottery number which is *normalized* to range from 0 to 1, where 0 corresponds to the lowest number and 1 the highest number. The authors have done this with a variable called `lotnum01`. To visualize this:

In [ ]:
sns.scatterplot(x='lotnum', y='lotnum01', data=colnomil)

**Question 3.3. Use `smf.ols` to tun a bivariate regression with `vietx` as the dependent variable and `lotnum01` as the independent variable.**

The slope you get from this should be the same as what you got question 3.2. This is reassuring: once we normalize the lottery number to be between 0 and 1, the slope tells us the change in the prediction of war attitudes as `lotnum01` goes up by one unit (i.e., the lowest to highest value). In other words, the change in predicted war attitude as we go from the lowest to highest lottery number isn't affected by whether we use the "raw number" or 0 to 1 version. 

From this analysis, we can be confident that having a lower draft number caused more anti-war attitudes. We know there is no selection bias because the lottery number was random, and so it should be unrelated to the potential outcomes. 

Some papers argue that the effect of having a low draft number can be used to estimate the causal effect of military service (on things like <a href="https://www.jstor.org/stable/2006669">lifetime wages</a>). Erikson and Stoker argue that for the case of political attitudes, it may make more sense to think about the lottery number itself as causing the change in attitudes, since having a low lottery number could induce fear/anxiety about getting drafted regardless of whether that actually happens.

**Question 3.4. One way to exlore this empirically is to run a multivariate regression with `vietx` as the dependent variable and `lotnum01` and `milserv` (which is equal to 1 for those who ended up serving in the military and 0 for those who did not) as independent variables. Use `smf.ols` to run this regression. (Recall for multivariate regression the "formula" argument should look like `DV ~ IV1 + IV2`.)**

A nice thing about using the 0 to 1 version of `lotnum` is we can compare the coefficients here, since `milserv` also ranges from 0 to 1. As we can see, the lottery number itself seemed to have a much larger impact on attitudes then whether the responded actually served in the military! Further, the coefficient on `lotnum01` is about the same as it was without controlling for `milserv`.

Recall an important assumption for us to try and estimate the effect of military service on an outcome (like war attitudes) is that our instrument (here, lottery number) only affects the outcome by changing the treatment (here, military service). This is called the *exclusion restriction.* While there is no direct way to test this assumption, the analysis here raises a red flag: not only does the lottery number seem to influence attitudes *holding military service fixed*, military service does not seem to help us predict war attitudes.

Another thing we might want to explore is how good of a job the lottery number does at predicting whether someone serves in the military. 

**Question 3.5. Use `smf.ols` to run a bivariate regression with `milserv` as the dependent variable and `lotnum01` as the independent variable.**

You should find a negative coefficient on `lotnum01`, but the magnitude is smaller than we might expect: going from the lowest to the highest lottery number only decreases the chance of getting drafted by less than 20%. (Compare this to the KIPP lottery where winning inceased the chance of enrolling by about 75%). 

Recall another important assumption if we want to estimate the effect of military service on war attitudes is that there is a "strong first stage". While there are no absolute rules here, one rule of thumb is that the F statistic (reported in the top right of the output) is at least 10, which it is not. 

So there are two reasons we should not use this data to estimate the effect of *military service* on war attitudes: (1) the exclusion restriction is probably not met, and (2) there is a "weak" first stage relationship.

However, we can still learn about the effect of *the threat of being drafted* on war attitues, and be confident that this effect represents a causal relationship. 

## Part 4: Is college worth it?

To get a bit of practice using two-stage least squares for an example where it is arguably more appropriate, let's turn to a paper by Berkeley's recent Nobel Laureate David Card. 

<a href="https://davidcard.berkeley.edu/papers/geo_var_schooling.pdf">This paper</a> studies a classic policy question in the social sciences: what is the impact of education on earnings? 

We can just run a regression with earnings as the DV and years of education as the IV, but there are some obvious reasons to be worried about selection bias: those who get more education are different on other dimensions (having the resources to attend college, intelligence, work ethic, etc.).

Card proposes using *growing up in a county with a college* as an instrument which affects how much education one gets. The core idea is that when college is closer, it is easier to attend, particularly for those with family obligations. 

Let's load up the data, which I'm getting from Scott Cunningham's <a href="https://mixtape.scunning.com/">"Causal Inference Mixtape"</a>, which is a great (and free!) book if you'd like to explore the topics we are covering with more technical detail.

In [ ]:
# read data
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
def read_data(file): 
    return pd.read_stata("https://raw.github.com/scunning1975/mixtape/master/" + file)
card = read_data("card.dta")
card

Fist let's look at the bivariate relationship between years of education (`educ`) and the natural log of wages (`lwage`). Economists and other social scientists often use the natural log of wages rather than the "raw" measure of wages for reasons we'll discuss in a few weeks. For now, the important thing to know is that we can interpret the coefficient as telling us the *percent increase* in wage as education goes up by one year.

In [ ]:
smf.ols("lwage ~ educ", data = card).fit().summary()

So, this implies that an extra year of education is associated with a 5% increase in wages. 

We can also control for some other variables we think might matter (gender, whether one is married, etc.).

In [ ]:
smf.ols("lwage ~ educ + exper + black + south + married + smsa", 
        data = card).fit().summary()

Now we get a somewhat *higher* coefficient, meaning that keeping these other variables fixed, another year of education is associated with an increase in wages of 7%. 

While we are going to keep focusing on education here, another result from this regression that could pique your interest is the coefficient on `black`, which indicates that even keeping fixed education and other relevant variables, black respondents earn about 16% less on average. Whether results like this are direct evidence of discrimination is a heavily debated topic.

Now let's turn to the "growing up near college" instrument. First we will use a variable `nearc4`, which is equal to 1 if the county one grew up in has a 4 year college and 0 otherwise. Let's run our "first stage" to see how this affects years of education.

In [ ]:
ols_fs_reg = smf.ols("educ ~ nearc4", data = card).fit()
ols_fs_reg.summary()

And our "reduced form" uses log wages as the dependent variable and growing up near a four year college as the depedent variable.

In [ ]:
ols_rf_reg = smf.ols("lwage ~ nearc4", data = card).fit()
ols_rf_reg.summary()

We can compute the two-stage least squares estimate of the causal effect of an extra year of college "by hand" by dividing the slope on `nearc4` in the regression with log wages as the dependent variable (analogous to `rho`) by the slope on `nearc4` in the regression with years of education as the dependent variable (analogous to `phi`):

In [ ]:
ols_rf_reg.params[1]/ols_fs_reg.params[1]

Now lets use the IV2SLS function from the linearmodels.iv library (already loaded up) to do this a bit more efficiently. 

If we want to estimate the causal effect of `treat` on `outcome` with instrument `instrument`, all of which are variables in a data frame `df`, the code we can use is:

`iv = IV2SLS.from_formula("outcome ~ 1 + [treat ~ instrument]", df).fit()`

Using the `.from_formula` allows us to specify our regression with a formula relatively similar to what we have been doing with `smf.ols`. Like before, we start our formula with `outcome ~`, but then there are two differences. First, we have to add a `1+` after the tilde; in short, this makes sure that we include a term for the y intercept (the $a$ or $\alpha$ from the slides). Second, we indicate what to use as the first stage by putting `[treat ~ instrument]` rather than just `treat`. 

To get output we then use the `summary` function:

`iv.summary`

Note that unlike with `smf.ols`, we don't add a `()` after the summary. 

Here is how it looks for a regresion with log wages as the outcome, education as the treatment, and growing up near a 4 year college as the instrument:

In [ ]:
iv_c4 = IV2SLS.from_formula("lwage ~ 1+[educ ~ nearc4 ]", card).fit()
iv_c4.summary

The coefficient is the same as we got above!

Another advantage to doing this with 2SLS is we can add other control variables (as discused in the book, to both the first stage and second stage). To do that, we use a similar line of code from before, but add `+V1 + V2 + ...` for the other control varibles. The treatment variable and the instrument still get entered as `[treat ~ instrument]`.

In [ ]:
#2SLS
iv_c4_mv = IV2SLS.from_formula("lwage ~ 1 + exper + black + south + married + smsa + [educ ~ nearc4 ]", 
                               data=card).fit()
iv_c4_mv.summary

Again, we estimate a strong causal effect of increasing years education (via growing up close to a 4 year college). In fact, the cofficient on `educ` is even higher than in the regular regression.

We can also look at the effect of growing up near a 2 year college, which is captured by the `nearc2` variable. 

**Question 4.1 Use `IV2SLS.from_formula` to run the regression with log wages as the outcome, education as the treatment, and `nearc2` as the instrument (with no control variables).**

**Question 4.2 Use `IV2SLS.from_formula` to run the regression with log wages as the outcome, education as the treatment, and `nearc2` as the instrument (with the same control variables as above).**

You should get a larger coefficient on `educ` in the regressions with `nearc2` as an instrument compared to `nearc4`. There are a few reasons this may be the case, but one possibility is that the "compliers" with the instruments are different. In other words, the kind of person induced to get more education by growing up near a two year college may be different than the kind of person induced to get more education by growing up near a four year college.

**Question 4.3 [Optional] Why by the compliers with the 2 year college instrument get a larger wage bump from going to college than those with the 4 year college instrument? Hint: you can go read the original paper for a discussion of this!**

**Question 4.4 [Optional] It is also possible that the exogeneity or exclusion restriction assumptions are not met here. Give a reason why these might not hold.**